In this tutorial, we build a simple matrix factorization model using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with TFRS. We can use this model to recommend movies for a given user.

### Import TFRS

First, install and import TFRS:

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [2]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

### Read the data

In [3]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

Shuffling and writing examples to C:\/Users/321ni/tensorflow_datasets/movielens/100k-ratings/0.1.0.incompleteJV4J9I/movielens-train.tfrecord


Dataset movielens downloaded and prepared to C:\Users\321ni\tensorflow_datasets\movielens\100k-ratings\0.1.0. Subsequent calls will reuse this data.


Shuffling and writing examples to C:\/Users/321ni/tensorflow_datasets/movielens/100k-movies/0.1.0.incomplete9NYPZK/movielens-train.tfrecord


Dataset movielens downloaded and prepared to C:\Users\321ni\tensorflow_datasets\movielens\100k-movies\0.1.0. Subsequent calls will reuse this data.


Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [4]:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

### Define a model

We can define a TFRS model by inheriting from `tfrs.Model` and implementing the `compute_loss` method:

In [5]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

Define the two models and the retrieval task.

In [6]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)


### Fit and evaluate it.

Create the model, train it, and generate predictions:



In [7]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index(movies.batch(100).map(model.movie_model), movies)

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Epoch 1/3


23/25 [==========================>...] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 2.4414e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0017 - factorized_top_k/top_10_categorical_accuracy: 0.0061 - factorized_top_k/top_50_categorical_accuracy: 0.0278 - factorized_top_k/top_100_categorical_accuracy: 0.0537 - loss: 34070.5078 - regularization_loss: 0.0000e+00 - total_loss: 34070.50 - ETA: 4s - factorized_top_k/top_1_categorical_accuracy: 1.2207e-04 - factorized_top_k/top_5_categorical_accuracy: 8.5449e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0031 - factorized_top_k/top_50_categorical_accuracy: 0.0142 - factorized_top_k/top_100_categorical_accuracy: 0.0297 - loss: 34084.7070 - regularization_loss: 0.0000e+00 - total_loss: 34084.70 - ETA: 5s - factorized_top_k/top_1_categorical_accuracy: 8.1380e-05 - factorized_top_k/top_5_categorical_accuracy: 7.3242e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0024 - factorized_top_k/top_50_categorical_accuracy: 

25/25 [==============================] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 9.1553e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0048 - factorized_top_k/top_50_categorical_accuracy: 0.0455 - factorized_top_k/top_100_categorical_accuracy: 0.1026 - loss: 34820.7762 - regularization_loss: 0.0000e+00 - total_loss: 34820.77 - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 9.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0048 - factorized_top_k/top_50_categorical_accuracy: 0.0458 - factorized_top_k/top_100_categorical_accuracy: 0.1035 - loss: 33922.4874 - regularization_loss: 0.0000e+00 - total_loss: 33922.48 - 10s 411ms/step - factorized_top_k/top_1_categorical_accuracy: 9.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0048 - factorized_top_k/top_50_categorical_accuracy: 0

23/25 [==========================>...] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 2.4414e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0022 - factorized_top_k/top_10_categorical_accuracy: 0.0076 - factorized_top_k/top_50_categorical_accuracy: 0.0762 - factorized_top_k/top_100_categorical_accuracy: 0.1633 - loss: 33218.0938 - regularization_loss: 0.0000e+00 - total_loss: 33218.09 - ETA: 10s - factorized_top_k/top_1_categorical_accuracy: 1.2207e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0020 - factorized_top_k/top_10_categorical_accuracy: 0.0079 - factorized_top_k/top_50_categorical_accuracy: 0.0729 - factorized_top_k/top_100_categorical_accuracy: 0.1653 - loss: 33204.3594 - regularization_loss: 0.0000e+00 - total_loss: 33204.359 - ETA: 14s - factorized_top_k/top_1_categorical_accuracy: 1.6276e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0020 - factorized_top_k/top_10_categorical_accuracy: 0.0074 - factorized_top_k/top_50_categorical_accuracy: 0.071

25/25 [==============================] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 1.4242e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0048 - factorized_top_k/top_10_categorical_accuracy: 0.0146 - factorized_top_k/top_50_categorical_accuracy: 0.1060 - factorized_top_k/top_100_categorical_accuracy: 0.2127 - loss: 32597.7250 - regularization_loss: 0.0000e+00 - total_loss: 32597.72 - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 1.4000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0049 - factorized_top_k/top_10_categorical_accuracy: 0.0146 - factorized_top_k/top_50_categorical_accuracy: 0.1062 - factorized_top_k/top_100_categorical_accuracy: 0.2129 - loss: 31770.0568 - regularization_loss: 0.0000e+00 - total_loss: 31770.05 - 13s 530ms/step - factorized_top_k/top_1_categorical_accuracy: 1.4000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0049 - factorized_top_k/top_10_categorical_accuracy: 0.0146 - factorized_top_k/top_50_categorical_accuracy: 0

23/25 [==========================>...] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0054 - factorized_top_k/top_10_categorical_accuracy: 0.0181 - factorized_top_k/top_50_categorical_accuracy: 0.1294 - factorized_top_k/top_100_categorical_accuracy: 0.2490 - loss: 32155.1387 - regularization_loss: 0.0000e+00 - total_loss: 32155.13 - ETA: 5s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0054 - factorized_top_k/top_10_categorical_accuracy: 0.0172 - factorized_top_k/top_50_categorical_accuracy: 0.1252 - factorized_top_k/top_100_categorical_accuracy: 0.2440 - loss: 32192.6885 - regularization_loss: 0.0000e+00 - total_loss: 32192.68 - ETA: 6s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0050 - factorized_top_k/top_10_categorical_accuracy: 0.0152 - factorized_top_k/top_50_categorical_accuracy: 0.1222 -

25/25 [==============================] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 3.7638e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0081 - factorized_top_k/top_10_categorical_accuracy: 0.0220 - factorized_top_k/top_50_categorical_accuracy: 0.1443 - factorized_top_k/top_100_categorical_accuracy: 0.2673 - loss: 31975.0675 - regularization_loss: 0.0000e+00 - total_loss: 31975.06 - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 3.7000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0081 - factorized_top_k/top_10_categorical_accuracy: 0.0220 - factorized_top_k/top_50_categorical_accuracy: 0.1443 - factorized_top_k/top_100_categorical_accuracy: 0.2676 - loss: 31164.0773 - regularization_loss: 0.0000e+00 - total_loss: 31164.07 - 11s 432ms/step - factorized_top_k/top_1_categorical_accuracy: 3.7000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0081 - factorized_top_k/top_10_categorical_accuracy: 0.0220 - factorized_top_k/top_50_categorical_accuracy: 0